<a href="https://colab.research.google.com/github/elilaird/CS83212-Advanced-Neural-Networks/blob/main/Lab2_Style_Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 2: Style Transfer

**Group Members:**
* Clay Harper
* Eli Laird


In [1]:
import tensorflow as tf
from tensorflow import keras

print(f'Tensorflow version: {tf.__version__}')
print(f'Keras version: {keras.__version__}')

import matplotlib.pyplot as plt
# import seaborn as sns
import numpy as np
import PIL
from PIL import Image
from io import BytesIO
import requests
#from tqdm import 
import copy

from tensorflow.keras.preprocessing import image
from tensorflow.keras import models, Model, Sequential
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Conv2D, Input, UpSampling2D, Conv2DTranspose, Layer
from tensorflow.keras.applications.vgg19 import preprocess_input as vgg_preprocess
from tensorflow.image import resize

Tensorflow version: 2.4.1
Keras version: 2.4.0


## VGG Manipulation

Here, we need to manipulate the given VGG code (courtesy of Justin Ledford) to make use of pooling layers or strided convolutions alternatively.  We chose to use strided convolutions because it is less computationally expensive.

In [2]:
# Load VGG
pre_trained_model = tf.keras.applications.VGG19(include_top=False,
                                                      weights='imagenet')

def vgg_layers(inputs, target_layer):
#     with tf.device('/cpu:0'):
#         # Block 1
#         x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(inputs)
#         if target_layer == 1:
#             return x
#         # Strides instead of maxpooling 
#         x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2', strides=2)(x)
#         # x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

#         # Block 2
#         x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
#         if target_layer == 2:
#             return x
#         x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2', strides=2)(x)
#         # x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

#         # Block 3
#         x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
#         if target_layer == 3:
#             return x
#         x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
#         x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
#         x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv4', strides=2)(x)
#         # x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

#         # Block 4
#         x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
#         if target_layer == 4:
#             return x
#         x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
#         x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
#         x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv4', strides=2)(x)
#         # x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

#         # Block 5
#         x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
#         return x
    # Block 1
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(inputs)
    if target_layer == 1:
        return x
    # Strides instead of maxpooling 
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2', strides=2)(x)
    # x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    if target_layer == 2:
        return x
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2', strides=2)(x)
    # x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    if target_layer == 3:
        return x
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv4', strides=2)(x)
    # x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    if target_layer == 4:
        return x
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv4', strides=2)(x)
    # x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    return x

def load_weights(trained_model, model):
    layer_names = [layer.name for layer in trained_model.layers]

    for layer in model.layers:
        b_name = layer.name.encode()
        if b_name in layer_names:
            layer.set_weights(trained_model.get_layer(b_name).get_weights())
            layer.trainable = False

def VGG19(trained_model, input_tensor=None, input_shape=None, target_layer=1):
    """
    VGG19, up to the target layer (1 for relu1_1, 2 for relu2_1, etc.)
    """
    if input_tensor is None:
        inputs = Input(shape=input_shape)
    else:
        inputs = Input(tensor=input_tensor, shape=input_shape)
    model = Model(inputs, vgg_layers(inputs, target_layer), name='vgg19', trainable=False)
    for layer in model.layers:
        layer.trainable = False
    load_weights(trained_model, model)
    return model

Create an encoder network from the pretrained VGG network 

In [3]:
target_layer = 3
vgg_model = VGG19(pre_trained_model, input_shape=(256, 256, 3), target_layer=target_layer)
vgg_model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 64, 64, 256)       295168    
Total params: 555,328
Trainable params: 0
Non-trainable params: 555,328
_______________________________________________________

## Decoder Network Architecture

In [4]:
def decoder_layers(inputs, layer):
#     with tf.device('/cpu:0'): #******change to gpu later
#         x = Conv2D(512, (3, 3), activation='relu', padding='same', name='decoder_block5_conv1')(inputs)
#         if layer == 5:
#             return x


#         #x = UpSampling2D((2, 2), name='decoder_block4_upsample')(x)
#         x = Conv2DTranspose(1, kernel_size=(4,4), padding='same', strides=(2,2), name='decoder_block4_2DTrans')(x)
#         x = Conv2D(512, (3, 3), activation='relu', padding='same', name='decoder_block4_conv4')(x)
#         x = Conv2D(512, (3, 3), activation='relu', padding='same', name='decoder_block4_conv3')(x)
#         x = Conv2D(512, (3, 3), activation='relu', padding='same', name='decoder_block4_conv2')(x)
#         x = Conv2D(512, (3, 3), activation='relu', padding='same', name='decoder_block4_conv1')(x)
#         if layer == 4:
#             return x

#         #x = UpSampling2D((2, 2), name='decoder_block3_upsample')(x)
#         x = Conv2DTranspose(1, kernel_size=(4,4), padding='same', strides=(2,2), name='decoder_block3_2DTrans')(x)
#         x = Conv2D(256, (3, 3), activation='relu', padding='same', name='decoder_block3_conv4')(x)
#         x = Conv2D(256, (3, 3), activation='relu', padding='same', name='decoder_block3_conv3')(x)
#         x = Conv2D(256, (3, 3), activation='relu', padding='same', name='decoder_block3_conv2')(x)
#         x = Conv2D(256, (3, 3), activation='relu', padding='same', name='decoder_block3_conv1')(x)
#         if layer == 3:
#             return x

#         #x = UpSampling2D((2, 2), name='decoder_block2_upsample')(x)
#         x = Conv2DTranspose(1, kernel_size=(4,4), padding='same', strides=(2,2), name='decoder_block2_2DTrans')(x)
#         x = Conv2D(128, (3, 3), activation='relu', padding='same', name='decoder_block2_conv2')(x)
#         x = Conv2D(128, (3, 3), activation='relu', padding='same', name='decoder_block2_conv1')(x)
#         if layer == 2:
#             return x

#         #x = UpSampling2D((2, 2), name='decoder_block1_upsample')(x)
#         x = Conv2DTranspose(1, kernel_size=(4,4), padding='same', strides=(2,2), name='decoder_block1_2DTrans')(x)
#         x = Conv2D(64, (3, 3), activation='relu', padding='same', name='decoder_block1_conv2')(x)
#         x = Conv2D(64, (3, 3), activation='relu', padding='same', name='decoder_block1_conv1')(x)
#         if layer == 1:
#             return x
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='decoder_block5_conv1')(inputs)
    if layer == 5:
        return x


    #x = UpSampling2D((2, 2), name='decoder_block4_upsample')(x)
    x = Conv2DTranspose(1, kernel_size=(4,4), padding='same', strides=(2,2), name='decoder_block4_2DTrans')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='decoder_block4_conv4')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='decoder_block4_conv3')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='decoder_block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='decoder_block4_conv1')(x)
    if layer == 4:
        return x

    #x = UpSampling2D((2, 2), name='decoder_block3_upsample')(x)
    x = Conv2DTranspose(1, kernel_size=(4,4), padding='same', strides=(2,2), name='decoder_block3_2DTrans')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='decoder_block3_conv4')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='decoder_block3_conv3')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='decoder_block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='decoder_block3_conv1')(x)
    if layer == 3:
        return x

    #x = UpSampling2D((2, 2), name='decoder_block2_upsample')(x)
    x = Conv2DTranspose(1, kernel_size=(4,4), padding='same', strides=(2,2), name='decoder_block2_2DTrans')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='decoder_block2_conv2')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='decoder_block2_conv1')(x)
    if layer == 2:
        return x

    #x = UpSampling2D((2, 2), name='decoder_block1_upsample')(x)
    x = Conv2DTranspose(1, kernel_size=(4,4), padding='same', strides=(2,2), name='decoder_block1_2DTrans')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='decoder_block1_conv2')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='decoder_block1_conv1')(x)
    if layer == 1:
        return x

# Covariance Matching

We decided to implement a covariance matching algorithm for the section where we got to choose one of the following:

* smoothing constraint
* style transfer with an image mask
* different covariance matching algorithm
* blend different styles in the reconstruction

The implementation we are using was proposed in https://openaccess.thecvf.com/content_ICCV_2019/papers/Lu_A_Closed-Form_Solution_to_Universal_Style_Transfer_ICCV_2019_paper.pdf

The closed form solution of the transformation matrix is:

$$T = \Sigma_c^{-1/2}(\Sigma_c^{1/2}\Sigma_s\Sigma_c^{1/2})^{1/2}\Sigma_c^{-1/2}$$

Where taking a matrix to a power is defined as:

$$A^\alpha = \Gamma\Lambda^\alpha\Gamma^T$$

Since the style image will always be the same in this architecture, we precompute the covariance of the style image and use it as a global variable.

In [5]:
# # sigma_s = np.cov(style_img)
# img = tf.io.read_file('./../data/style.jpg')
# img = tf.image.decode_image(img, channels=3).numpy()
# # img = tf.image.convert_image_dtype(img, tf.float32)
# img =

In [29]:
def tf_cov(self, x):
#     with tf.device('/CPU:0'):
#         x = x - tf.expand_dims(tf.reduce_mean(x, axis=1), 1)
#         fact = tf.cast(tf.shape(x)[1] - 1, tf.float32)
#         b = tf.cast(tf.math.conj(tf.transpose(x)), tf.float32)
#         return tf.matmul(x, b) / fact

    return tf.matmul(x, tf.transpose(x))

In [33]:
class TFTest(Layer):
    def __init__(self, sigma_s, **kwargs):
        super(TFTest, self).__init__()
        self.sigma_s = sigma_s
        
    def matrix_to_power(self, matrix, power):
        e_val, e_vec = tf.linalg.eigh(matrix)
        lambda_ = tf.linalg.diag(e_val**power)

        return e_vec@lambda_@tf.transpose(e_vec)
    
    def call(self, content_img):
        # Reshape/squeeze to be the correct size 
        content_mat = tf.squeeze(content_img)
        squeezed_shape = tf.shape(content_mat)
        channels = tf.shape(content_mat)[-1]
        content_mat = tf.reshape(content_mat, (channels, -1))
        
        sigma_c = tf.matmul(content_mat, tf.transpose(content_mat))
        
        sigma_c_neg_half = self.matrix_to_power(sigma_c, -1/2)
        sigma_c_half = self.matrix_to_power(sigma_c, 1/2)
        
        inner_prod = tf.matmul(tf.matmul(sigma_c_half, self.sigma_s), sigma_c_half)
        inner = self.matrix_to_power(inner_prod, 1/2)
        
        transform_mat = tf.matmul(tf.matmul(sigma_c_neg_half, inner), sigma_c_neg_half)
        
        transformed = tf.matmul(transform_mat, content_mat)
        # add back batch dim and reshape
        transformed = tf.reshape(transformed, content_mat)
        return tf.expand_dims(transform_mat, axis=0)

In [17]:
# sigma_s = np.cov(style_img)
# img = tf.io.read_file('./../data/style.jpg')
# img = tf.image.decode_image(img, channels=3)
# img = tf.image.convert_image_dtype(img, tf.float32)


def matrix_to_power(matrix, power):
    # Assuming real symmetric
    e_val, e_vec = np.linalg.eigh(matrix)
    lambda_ = np.diag(e_val**power)

    return e_vec@lambda_@e_vec.T

def new_transform(content_img, sigma_s):
    print(type(content_img))
    print(f'Received shape: {content_img.shape}')
    
    channel_dim = sigma_s.shape[0]
    print(f'Channel dim: {channel_dim}')
    
    content_img = np.squeeze(content_img).reshape((channel_dim, -1))
    print(f'content img shape: {content_img.shape}')
    sigma_c = np.cov(content_img)
    print(sigma_c.shape)
    print(sigma_c)
#     sigma_s = np.cov(style_img)


    simga_c_neg_half = matrix_to_power(sigma_c, -1/2)
    sigma_c_half = matrix_to_power(sigma_c, 1/2)

    inner = matrix_to_power(sigma_c_half@sigma_s@sigma_c_half, 1/2)

    transform_mat = simga_c_neg_half@inner@simga_c_neg_half
    return transform_mat@content_img

class CovarianceMatching(Layer):
    def __init__(self, sigma_s, **kwargs):
        super(CovarianceMatching, self).__init__()
        self.sigma_s = sigma_s
        
    def call(self, xin):
        xout = tf.py_function(new_transform, 
                           (xin, self.sigma_s),
                           'float32',
                           name='cov_matching')
        xout = K.stop_gradient(xout) # explicitly set no grad
        xout.set_shape(xin.shape) # explicitly set output shape
        return xout
    
    def compute_output_shape(self, sin):
        return sin.shape
    

class TFCovarianceMatching(Layer):
    def __init__(self, sigma_s, **kwargs):
        super(TFCovarianceMatching, self).__init__()
        self.sigma_s = sigma_s
        
    #from https://stackoverflow.com/questions/47709854/how-to-get-covariance-matrix-in-tensorflow
    def tf_cov(self, x):
        with tf.device('/CPU:0'):
            x = x - tf.expand_dims(tf.reduce_mean(x, axis=1), 1)
            fact = tf.cast(tf.shape(x)[1] - 1, tf.float32)
            b = tf.cast(tf.math.conj(tf.transpose(x)), tf.float32)
            return tf.matmul(x, b) / fact


    def matrix_to_power(self, matrix, power):
        with tf.device('/CPU:0'):
            # Assuming real symmetric
            e_val, e_vec = tf.linalg.eigh(matrix)
            lambda_ = tf.linalg.diag(e_val**power)

            return e_vec@lambda_@tf.transpose(e_vec)

    def call(self, content_img):
        with tf.device('/CPU:0'):
            h,w,c = content_img.shape[1:]
            content_img = tf.squeeze(content_img)
            content_img = tf.reshape(content_img, [h,w,c])
            content_img = tf.reshape(content_img, [c, h * w]) 

            sigma_c = self.tf_cov(content_img)

            sigma_c_neg_half = self.matrix_to_power(sigma_c, -1/2)
            sigma_c_half = self.matrix_to_power(sigma_c, 1/2)
            inner = self.matrix_to_power(sigma_c_half@self.sigma_s@sigma_c_half, 1/2)
            transform_mat = sigma_c_neg_half@inner@sigma_c_neg_half

            out = transform_mat@content_img
            out = tf.reshape(out, [c,h,w])
            out = tf.expand_dims(tf.reshape(out, [h,w,c]),0)

            return out

# tf.py_function

## Encoder-Decoder Architecture

In [60]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.optimizers import Adam

LAMBDA=1

def l2_loss(x):
    return K.sum(K.square(x)) / 2

class EncoderDecoder:
    def __init__(self, trained_model, input_shape=(256, 256, 3), target_layer=5, style_path='../data/style.jpg', decoder_path=None):
        self.input_shape = input_shape
        self.target_layer = target_layer
        self.trained_model = trained_model

    

        self.encoder = VGG19(self.trained_model, input_shape=self.input_shape, target_layer=target_layer)
        
        # Load style image and add batch dim
#         self.style_image = img_to_array(load_img(style_path, target_size=self.input_shape))
        img = tf.io.read_file(style_path)
        img = tf.image.decode_image(img, channels=3)
        img = vgg_preprocess(img)
        self.style_image = resize(img, self.encoder.input_shape[1:3])
        style_latent = self.encoder(tf.expand_dims(self.style_image, axis=0)).numpy()
        self.sigma_s = np.cov(np.squeeze(style_latent).reshape(style_latent.shape[-1], -1))
        
    
        if decoder_path:
            self.decoder = load_model(decoder_path)
        else:
            self.decoder = self.create_decoder(target_layer)
            
        # Create the complete model (encoder -> covar_match -> decoder)
#         transformed_tensor = CovarianceMatching(self.sigma_s, name='covar_matching')(self.encoder.output)
#         transformed_tensor = TFCovarianceMatching(self.sigma_s, name='covar_matching')(self.encoder.output)
        
        decoder_output = self.decoder(self.encoder.output)

        self.model = Model(self.encoder.input, decoder_output)
        
        self.loss = self.create_loss_fn(self.encoder)

        adam = Adam(learning_rate=1e-4)
        self.model.compile(adam, self.loss)

    def create_loss_fn(self, encoder):
        def get_encodings(inputs):
            encoder = VGG19(self.trained_model, inputs, self.input_shape, self.target_layer)
            return encoder.output

        def loss(img_in, img_out):
            encoding_in = get_encodings(img_in)
            encoding_out = get_encodings(img_out)
            return l2_loss(img_out - img_in) + \
                  LAMBDA*l2_loss(encoding_out - encoding_in)
        return loss

    def summary(self):
        self.model.summary()

    def create_decoder(self, target_layer):
        inputs = Input(shape=self.encoder.output.shape[1:])
        layers = decoder_layers(inputs, target_layer)
        output = Conv2D(3, (3, 3), activation='relu', padding='same',
                        name='decoder_out')(layers)
        decoder = Model(inputs, output, name='decoder_%s' % target_layer)
        print(decoder.summary())
        return decoder
    
#     def create_transform_layer(self):
#         inputs = Input(shape=self.encoder.output_shape[1:])
#         output = new_transform(inputs, tf.constant(self.sigma_style, dtype=tf.float32))
#         return Model(inputs, output)
    
#     def load_style_img(self, img_path):
#         img = image.load_img(img_path)
#         img = image.img_to_array(img)
#         img = resize(img, (self.input_shape[0], self.input_shape[1]))
#         #img = tf.reshape(img, [self.input_shape[2], self.input_shape[0], self.input_shape[1]])
#         return tf.expand_dims(img, axis=0)
    
    def process_style_img(self, img):
        out = self.encoder(img)  
        out = tf.squeeze(out)
        out = tf.reshape(out, [out.shape[2], out.shape[0] * out.shape[1]])
#         return 
#         return tf_cov(out)
    
    def resize_for_transform(self,img):
        img = tf.squeeze(img)

        img = tf.reshape(img, [self.input_shape[2], self.input_shape[0], self.input_shape[1]])
        return img
    

    def export_decoder(self):
        self.decoder.save('decoder_%s.h5' % self.target_layer)


## Train Two Decoders 

Decoders will be created based on the outputs of 2 different layers in the encoder model.

In [61]:
encoder_decoder = EncoderDecoder(pre_trained_model, target_layer=target_layer)
encoder_decoder.summary()

Model: "decoder_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 64, 64, 256)]     0         
_________________________________________________________________
decoder_block5_conv1 (Conv2D (None, 64, 64, 512)       1180160   
_________________________________________________________________
decoder_block4_2DTrans (Conv (None, 128, 128, 1)       8193      
_________________________________________________________________
decoder_block4_conv4 (Conv2D (None, 128, 128, 512)     5120      
_________________________________________________________________
decoder_block4_conv3 (Conv2D (None, 128, 128, 512)     2359808   
_________________________________________________________________
decoder_block4_conv2 (Conv2D (None, 128, 128, 512)     2359808   
_________________________________________________________________
decoder_block4_conv1 (Conv2D (None, 128, 128, 512)     23

In [62]:
import tensorflow_datasets as tfds
from tensorflow.keras.applications.vgg19 import preprocess_input as vgg_preprocess
print(f'{tfds.__version__}')

4.2.0+nightly


In [101]:
%%time
# Create a dataset to train on (imagenette)
data_loader = tfds.load("imagenette", download=True)

CPU times: user 45.3 ms, sys: 2.01 ms, total: 47.3 ms
Wall time: 46 ms


In [102]:
data_loader

{'train': <PrefetchDataset shapes: {image: (None, None, 3), label: ()}, types: {image: tf.uint8, label: tf.int64}>,
 'validation': <PrefetchDataset shapes: {image: (None, None, 3), label: ()}, types: {image: tf.uint8, label: tf.int64}>}

In [103]:
train_ds, test_ds = data_loader['train'], data_loader['validation']
train_ds

<PrefetchDataset shapes: {image: (None, None, 3), label: ()}, types: {image: tf.uint8, label: tf.int64}>

In [104]:
from tensorflow.image import resize
train_ds, test_ds = data_loader['train'], data_loader['validation']

# Add batches and preprocessing 
BATCH_SIZE=1
n_train_observations = train_ds.cardinality().numpy()
n_test_observations = test_ds.cardinality().numpy()

steps_per_epoch = n_train_observations//BATCH_SIZE + n_train_observations%BATCH_SIZE
validation_steps = n_test_observations//BATCH_SIZE + n_test_observations%BATCH_SIZE

def preprocess(observation):
    img = observation['image']
    label = observation['label']
    
    # Resize to target shape
    processed_img = resize(img, encoder_decoder.input_shape[:2])
    
    # vgg preprocess
    processed_img = vgg_preprocess(processed_img)
    
#     return processed_img, label
    return processed_img, processed_img #for encoder/decoder loss

# train_ds = train_ds.map(
#     lambda image, label: (preprocess(image), label)).shuffle(1000).batch(BATCH_SIZE).repeat()
# test_ds = test_ds.map(
#     lambda image, label: (preprocess(image), label)).shuffle(1000).batch(BATCH_SIZE).repeat()

train_ds = train_ds.map(
    lambda observation: preprocess(observation)).shuffle(1000).batch(BATCH_SIZE).repeat(1000)
test_ds = test_ds.map(
    lambda observation: preprocess(observation)).shuffle(1000).batch(BATCH_SIZE).repeat(1000)


# train_ds = train_ds.batch(BATCH_SIZE).map(
#     lambda image, label: (preprocess(image), label)).shuffle(1000).repeat()
# test_ds = test_ds.batch(BATCH_SIZE).map(
#     lambda image, label: (preprocess(image), label)).shuffle(1000).repeat()

print(f'Train observations: {n_train_observations}')
print(f'Test observations: {n_test_observations}')

Train observations: 9469
Test observations: 3925


In [105]:
train_ds

<RepeatDataset shapes: ((None, 256, 256, 3), (None, 256, 256, 3)), types: (tf.float32, tf.float32)>

In [106]:
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, EarlyStopping, ModelCheckpoint


# This needs to be in scope where model is defined
# class OutputPreview(Callback):
#     def __init__(self, model, test_img_path, increment, preview_dir_path):
#         test_img = image.load_img(test_img_path)
#         test_target = image.img_to_array(test_img)
#         test_target = resize(test_target, (256,256)) #imresize(test_img, (256, 256, 3))
#         #test_target = image.img_to_array(test_img)
#         test_target = np.expand_dims(test_target, axis=0)
#         self.test_img = test_target
#         self.model = model

#         self.preview_dir_path = preview_dir_path

#         self.increment = increment
#         self.iteration = 0

#     def on_batch_end(self, batch, logs={}):
#         if (self.iteration % self.increment == 0):
#             output_img = self.model.predict(self.test_img)[0]
#             fname = '%d.jpg' % self.iteration
#             out_path = os.path.join(self.preview_dir_path, fname)
#             imsave(out_path, output_img)

#         self.iteration += 1

In [108]:
history = encoder_decoder.model.fit(train_ds,
                                    epochs=1000,
                                    steps_per_epoch=steps_per_epoch,
                                    verbose=1)

Epoch 1/1000


TypeError: 'NoneType' object is not callable

In [107]:
callbacks = [
    ModelCheckpoint(filepath='./encoder_decoder.h5'),
    ReduceLROnPlateau(monitor='val_loss', factor=.1, patience=5, min_lr=1e-6),
    EarlyStopping(patience=7),
#     OutputPreview(encoder_decoder, '../data/doge.jpg', 5000, './preview-%d' % target_layer)
]

history = encoder_decoder.model.fit(train_ds, validation_data=test_ds,
                                    epochs=1000,
                                    callbacks=callbacks,
                                    steps_per_epoch=steps_per_epoch,
                                    validation_steps=validation_steps,
                                    verbose=1)

Epoch 1/1000


TypeError: 'NoneType' object is not callable

In [26]:
encoder_decoder.model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 64, 64, 256)       295168    
_________________________________________________________________
covariance_matching (Covaria (None, 64, 64, 256)       0     

In [85]:
(ds_train, ds_test), ds_info = tfds.load(
    'imagenette',
    split=['train', 'validation'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [86]:
# Add batches and preprocessing 
BATCH_SIZE=1
n_train_observations = ds_info.splits['train'].num_examples
n_test_observations = ds_info.splits['validation'].num_examples

steps_per_epoch = n_train_observations//BATCH_SIZE + n_train_observations%BATCH_SIZE
validation_steps = n_test_observations//BATCH_SIZE + n_test_observations%BATCH_SIZE
print(f'Train observations: {n_train_observations}')
print(f'Test observations: {n_test_observations}')
print(ds_info.splits['validation'].num_examples)

Train observations: 9469
Test observations: 3925
3925


In [72]:
def testing(img):
    
    # Resize to target shape
    processed_img = resize(img, encoder_decoder.input_shape[:2])
    
    # vgg preprocess
    processed_img = vgg_preprocess(processed_img)
    
    return processed_img

ds_train = ds_train.map(lambda image, label: (testing(image), label))
ds_train = ds_train.shuffle(n_train_observations)
ds_train = ds_train.batch(1)
# ds = ds.repeat()
ds_train

<BatchDataset shapes: ((None, 256, 256, 3), (None,)), types: (tf.float32, tf.int64)>

In [73]:
ds_test = ds_test.map(lambda image, label: (testing(image), label))
ds_test = ds_test.shuffle(n_test_observations)
ds_test = ds_test.batch(1)
# ds = ds.repeat()
ds_test

<BatchDataset shapes: ((None, 256, 256, 3), (None,)), types: (tf.float32, tf.int64)>

In [59]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
inputs = encoder_decoder.encoder.output
# flattened = GlobalAveragePooling2D()(inputs)
# covar = TFTest(sigma_s=tf.convert_to_tensor(np.cov(np.random.rand(256, 256)), dtype='float32'))(inputs)

net = Dense(10)(covar)

mod = Model(encoder_decoder.encoder.inputs, net)
mod.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)
mod.summary()

(None, 64, 64, 256)
<unknown>


TypeError: in user code:

    <ipython-input-58-aa07504045c2>:19 call  *
        content_mat = tf.reshape(content_mat, (channels, -1))
    /Users/clayharper/anaconda/envs/ml_spring_21/lib/python3.6/site-packages/tensorflow/python/util/dispatch.py:201 wrapper  **
        return target(*args, **kwargs)
    /Users/clayharper/anaconda/envs/ml_spring_21/lib/python3.6/site-packages/tensorflow/python/ops/array_ops.py:195 reshape
        result = gen_array_ops.reshape(tensor, shape, name)
    /Users/clayharper/anaconda/envs/ml_spring_21/lib/python3.6/site-packages/tensorflow/python/ops/gen_array_ops.py:8378 reshape
        "Reshape", tensor=tensor, shape=shape, name=name)
    /Users/clayharper/anaconda/envs/ml_spring_21/lib/python3.6/site-packages/tensorflow/python/framework/op_def_library.py:525 _apply_op_helper
        raise err
    /Users/clayharper/anaconda/envs/ml_spring_21/lib/python3.6/site-packages/tensorflow/python/framework/op_def_library.py:515 _apply_op_helper
        preferred_dty

In [83]:

# mod = Model(encoder_decoder.encoder.input, encoder_decoder.decoder(encoder_decoder.encoder.output))
mod = Model(encoder_decoder.encoder.input, encoder_decoder.encoder.output)
mod.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=encoder_decoder.loss,
)

mod.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 64, 64, 256)       295168    
Total params: 555,328
Trainable params: 0
Non-trainable params: 555,328
_____________________________________________________

In [84]:
mod.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

Epoch 1/6


ValueError: in user code:

    /Users/clayharper/anaconda/envs/ml_spring_21/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    <ipython-input-60-581b4ad750c2>:54 loss  *
        encoding_in = get_encodings(img_in)
    <ipython-input-60-581b4ad750c2>:50 get_encodings  *
        encoder = VGG19(self.trained_model, inputs, self.input_shape, self.target_layer)
    <ipython-input-2-6d8ba0bbdb9f>:97 VGG19  *
        model = Model(inputs, vgg_layers(inputs, target_layer), name='vgg19', trainable=False)
    <ipython-input-2-6d8ba0bbdb9f>:44 vgg_layers  *
        x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(inputs)
    /Users/clayharper/anaconda/envs/ml_spring_21/lib/python3.6/site-packages/tensorflow/python/keras/engine/base_layer.py:952 __call__  **
        input_list)
    /Users/clayharper/anaconda/envs/ml_spring_21/lib/python3.6/site-packages/tensorflow/python/keras/e

In [75]:
encoder_decoder.model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

Epoch 1/6


ValueError: in user code:

    /Users/clayharper/anaconda/envs/ml_spring_21/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    <ipython-input-60-581b4ad750c2>:54 loss  *
        encoding_in = get_encodings(img_in)
    <ipython-input-60-581b4ad750c2>:50 get_encodings  *
        encoder = VGG19(self.trained_model, inputs, self.input_shape, self.target_layer)
    <ipython-input-2-6d8ba0bbdb9f>:97 VGG19  *
        model = Model(inputs, vgg_layers(inputs, target_layer), name='vgg19', trainable=False)
    <ipython-input-2-6d8ba0bbdb9f>:44 vgg_layers  *
        x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(inputs)
    /Users/clayharper/anaconda/envs/ml_spring_21/lib/python3.6/site-packages/tensorflow/python/keras/engine/base_layer.py:952 __call__  **
        input_list)
    /Users/clayharper/anaconda/envs/ml_spring_21/lib/python3.6/site-packages/tensorflow/python/keras/e

In [47]:
# model = tf.keras.models.Sequential([
#   tf.keras.layers.Flatten(input_shape=(256, 256, 3)),
#   tf.keras.layers.Dense(5,activation='relu'),
#   tf.keras.layers.Dense(10)
# ])

from tensorflow.keras.layers import Dense

inputs = Input(shape=(256, 256, 3))
net = Conv2D(5, (3,3))(inputs)
net = tf.keras.layers.GlobalAveragePooling2D()(net)
net = Dense(10)(net)

model = Model(inputs, net)

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, 256, 256, 3)]     0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 254, 254, 5)       140       
_________________________________________________________________
global_average_pooling2d_3 ( (None, 5)                 0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                60        
Total params: 200
Trainable params: 200
Non-trainable params: 0
_________________________________________________________________


In [32]:
model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

Epoch 1/6
3678/9469 [==========>...................] - ETA: 30s - loss: 5.5712 - sparse_categorical_accuracy: 0.1741

KeyboardInterrupt: 

In [20]:
ds, ds_info = tfds.load(
    'imagenette',
    split='train',
    as_supervised=True,  # returns `(img, label)` instead of dict(image=, ...)
    with_info=True,
)

ds

<PrefetchDataset shapes: ((None, None, 3), ()), types: (tf.uint8, tf.int64)>

In [21]:
def testing(img):
    
    # Resize to target shape
    processed_img = resize(img, encoder_decoder.input_shape[:2])
    
    # vgg preprocess
    processed_img = vgg_preprocess(processed_img)
    
    return processed_img

ds = ds.map(lambda image, label: (testing(image), label))
ds = ds.shuffle(n_train_observations)
ds = ds.batch(1)
# ds = ds.repeat()
ds

<BatchDataset shapes: ((None, 256, 256, 3), (None,)), types: (tf.float32, tf.int64)>

In [296]:
ds.keys()

AttributeError: 'BatchDataset' object has no attribute 'keys'

In [295]:
encoder_decoder.model.fit(ds)

ValueError: in user code:

    /Users/clayharper/anaconda/envs/ml_spring_21/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    <ipython-input-42-e8087fa9d216>:46 loss  *
        encoding_in = get_encodings(img_in)
    <ipython-input-42-e8087fa9d216>:42 get_encodings  *
        encoder = VGG19(self.trained_model, inputs, self.input_shape, self.target_layer)
    <ipython-input-2-e2ef8083919b>:61 VGG19  *
        model = Model(inputs, vgg_layers(inputs, target_layer), name='vgg19', trainable=False)
    <ipython-input-2-e2ef8083919b>:8 vgg_layers  *
        x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(inputs)
    /Users/clayharper/anaconda/envs/ml_spring_21/lib/python3.6/site-packages/tensorflow/python/keras/engine/base_layer.py:952 __call__  **
        input_list)
    /Users/clayharper/anaconda/envs/ml_spring_21/lib/python3.6/site-packages/tensorflow/python/keras/en

In [22]:
(ds_train, ds_test), ds_info = tfds.load(
    'imagenette',
    split=['train', 'validation'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [23]:
ds_train

<_OptionsDataset shapes: ((None, None, 3), ()), types: (tf.uint8, tf.int64)>

In [24]:
def normalize_img(img, label):    
    # Resize to target shape
    processed_img = resize(img, encoder_decoder.input_shape[:2])
    
    # vgg preprocess
    processed_img = vgg_preprocess(processed_img)
    
    return processed_img, label

In [320]:

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

In [321]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

In [322]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(256, 256, 3)),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(10)
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 196608)            0         
_________________________________________________________________
dense_7 (Dense)              (None, 128)               25165952  
_________________________________________________________________
dense_8 (Dense)              (None, 10)                1290      
Total params: 25,167,242
Trainable params: 25,167,242
Non-trainable params: 0
_________________________________________________________________


In [323]:
model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

Epoch 1/6
74/74 [==============================] - ETA: 0s - loss: 5935.3741 - sparse_categorical_accuracy: 0.2487

ValueError: When providing an infinite dataset, you must specify the number of steps to run (if you did not intend to create an infinite dataset, make sure to not call `repeat()` on the dataset).

In [97]:
train_ds

<RepeatDataset shapes: ((None, 256, 256, 3), (None,)), types: (tf.float32, tf.int64)>

In [94]:
train_ds.take(1)

<TakeDataset shapes: ((None, 256, 256, 3), (None,)), types: (tf.float32, tf.int64)>

In [98]:
count = 1

for element in train_ds.take(2):
    print(element)
    if count == 1:
        break

(<tf.Tensor: shape=(1, 256, 256, 3), dtype=float32, numpy=
array([[[[115.228966  , 122.38897   ,  42.48797   ],
         [106.4399    , 113.59991   ,  34.542656  ],
         [102.21725   , 108.49053   ,  31.843437  ],
         ...,
         [122.428185  , 119.80694   ,  85.49969   ],
         [104.95162   , 100.83819   ,  66.93719   ],
         [113.041466  , 107.45147   ,  73.55047   ]],

        [[105.76412   , 112.549126  ,  32.648125  ],
         [104.053185  , 110.52178   ,  31.780937  ],
         [105.04928   , 110.580376  ,  35.491875  ],
         ...,
         [118.60787   , 116.95928   ,  82.429375  ],
         [105.20553   , 102.49053   ,  68.21453   ],
         [109.135216  , 106.42022   ,  72.14422   ]],

        [[109.416466  , 115.40069   ,  35.85125   ],
         [112.74459   , 117.96319   ,  40.999687  ],
         [123.07662   , 127.611626  ,  54.02703   ],
         ...,
         [110.853966  , 111.56866   ,  76.667656  ],
         [102.666466  , 103.07647   ,  68.17547

In [81]:
import sys
import os

from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.callbacks import Callback
#from scipy.misc import imresize, imsave DEPRACATED
#from cv2 import resize
from tensorflow.image import resize
import numpy as np
import tensorflow_datasets as tfds


#from model import EncoderDecoder
#from util import count_num_samples

DATASET_DIR = '/Volumes/Eli\'s Disk/datasets'
TRAIN_PATH = os.path.join(DATASET_DIR, 'imagenette2', 'train')
VAL_PATH = os.path.join(DATASET_DIR, 'imagenette2', 'val')
TARGET_SIZE = (256, 256)
BATCH_SIZE = 1
epochs = 2

print(TRAIN_PATH)
datagen = ImageDataGenerator()
'''train_gen = datagen.flow_from_directory(TRAIN_PATH, target_size=TARGET_SIZE,
                                  batch_size=BATCH_SIZE, class_mode=None)

val_gen = datagen.flow_from_directory(VAL_PATH, target_size=TARGET_SIZE,
                                  batch_size=BATCH_SIZE, class_mode=None)'''

(train,val), info = tfds.load(name='imagenette', 
                       batch_size=BATCH_SIZE, 
                       split=['train','validation'], 
                       with_info=True,
                       download=True)


def create_gen(ds):
    def tuple_gen():
        for i in ds:

            # (X, y)
            
            img = tf.keras.applications.vgg19.preprocess_input(i['image'])
            img = tf.image.resize(img, TARGET_SIZE)
            yield (img,img)

    return tuple_gen()

train_gen = create_gen(train)
val_gen = create_gen(val)
'''train_gen = create_gen(TRAIN_PATH, TARGET_SIZE, BATCH_SIZE)
val_gen = create_gen(VAL_PATH, TARGET_SIZE, BATCH_SIZE)'''

# This needs to be in scope where model is defined
class OutputPreview(Callback):
    def __init__(self, model, test_img_path, increment, preview_dir_path):
        test_img = image.load_img(test_img_path)
        test_target = image.img_to_array(test_img)
        test_target = resize(test_target, (256,256)) #imresize(test_img, (256, 256, 3))
        #test_target = image.img_to_array(test_img)
        test_target = np.expand_dims(test_target, axis=0)
        self.test_img = test_target
        self.model = model

        self.preview_dir_path = preview_dir_path

        self.increment = increment
        self.iteration = 0

    def on_batch_end(self, batch, logs={}):
        if (self.iteration % self.increment == 0):
            output_img = self.model.predict(self.test_img)[0]
            fname = '%d.jpg' % self.iteration
            out_path = os.path.join(self.preview_dir_path, fname)
            imsave(out_path, output_img)

        self.iteration += 1


num_samples = 9469 #count_num_samples(TRAIN_PATH)
steps_per_epoch = num_samples // BATCH_SIZE

target_layer = 3 #int(sys.argv[1])

encoder_decoder = EncoderDecoder(trained_model=pre_trained_model, target_layer=target_layer)
encoder_decoder.summary()

callbacks = [OutputPreview(encoder_decoder, '../data/doge.jpg', 5000, './preview-%d' % target_layer)]


/Volumes/Eli's Disk/datasets/imagenette2/train
Model: "model_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_51 (InputLayer)        [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 64, 64, 256)       295168    
_________________________________________________________________
model_26 (F

In [84]:
encoder_decoder.model.fit(train_gen, steps_per_epoch=steps_per_epoch,
        epochs=epochs, callbacks=callbacks, validation_data=val_gen)
encoder_decoder.export_decoder()

Epoch 1/2


ValueError: in user code:

    /Users/eli/anaconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    <ipython-input-10-e471bb66f049>:40 loss  *
        encoding_in = get_encodings(img_in)
    <ipython-input-10-e471bb66f049>:36 get_encodings  *
        encoder = VGG19(self.trained_model, inputs, self.input_shape, self.target_layer)
    <ipython-input-51-e2ef8083919b>:61 VGG19  *
        model = Model(inputs, vgg_layers(inputs, target_layer), name='vgg19', trainable=False)
    <ipython-input-51-e2ef8083919b>:8 vgg_layers  *
        x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(inputs)
    /Users/eli/anaconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/keras/engine/base_layer.py:952 __call__  **
        input_list)
    /Users/eli/anaconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/keras/engine/base_layer.py:1091 _functional_constructi

# Covariance matching

## Dr. Larson 

Reusing code to test if mine works

In [ ]:
from scipy.ndimage import convolve
from matplotlib import pyplot as plt
%matplotlib inline
channels = 5 # number of channels to simulate
hw = 256 # height and width dimensions

# Create some Activation tensors
A1 = np.random.randn(channels,hw,hw)
A2 = np.random.rand(channels,hw,hw)

# make A1 low frequency  
filt = np.ones((3,6,6))
filt = filt / np.sum(filt)
A1 = convolve(A1,filt)
A1 = A1 - np.mean(A1)

# make A2 high frequency
filt = np.ones((2,3,3))
filt = filt / np.sum(filt)
filt[:,:,0] = filt[:,:,0]*(-1)
filt[:,:,1] = 0
A2 = convolve(A2,filt)
A2 = A2 - np.mean(A2)

# plot each tensor, average along channel
def show_active(A,title=''):
    plt.imshow(np.mean(A,0))
    plt.title(title)

n_plots = min(channels,10)
plt.figure(figsize=(15,5))   
for i in range(n_plots):
    plt.subplot(1,n_plots,i+1)
    plt.imshow(A1[i,:,:])
    plt.title(f'A1 at {i}')
    
n_plots = min(channels,10)
plt.figure(figsize=(15,5))   
for i in range(n_plots):
    plt.subplot(1,n_plots,i+1)
    plt.imshow(A2[i,:,:])
    plt.title(f'A2 at {i}')
    
plt.figure()
plt.subplot(1,2,1)
show_active(A1,'A1 Mean over channels')
plt.subplot(1,2,2)
show_active(A2,'A2 Mean over channels')

In [ ]:
# calculate gram matrix (vectorized spatially) and show it
# for each activations
def gram(x):
    tmp = x.reshape((channels,hw*hw))
    return tmp @ tmp.T / (channels*hw*hw)

def show_gram(A,title=''):
    plt.figure()
    G = gram(A)
    plt.subplot(1,2,2)
    plt.imshow(G)
    plt.title('Gram of '+title)
    plt.subplot(1,2,1)
    show_active(A,title)
    return G
    

G_A1 = show_gram(A1,'A1')
G_A2 = show_gram(A2,'A2')

In [ ]:
def whiten_cov(A):
    Avec = A.reshape((channels,hw*hw))
    AA = Avec @ Avec.T
    # notice that the SVD is of the covariance matrix
    u,s,vt = np.linalg.svd(AA,full_matrices=False)
    
    A_white = ((u @ np.diag(s**(-0.5)) @ vt) @ Avec) # no usage of the singular values here
    return A_white, (u, s, vt), Avec
    

A1_white, A1SVDcov, A1vec = whiten_cov(A1)
A2_white, A2SVDcov, A2vec = whiten_cov(A2)
    
u1, s1, _ = A1SVDcov
u2, s2, _ = A2SVDcov

# -------A1-------
# Apply inverse reconstruction to get colored version
A1_mixed_color = u2 @ np.diag(s2**(0.5)) @ u2.T @ A1_white
A1_mixed_color = A1_mixed_color.reshape((channels,hw,hw))

# -------A2-------
# Apply inverse reconstruction to get colored version
A2_mixed_color = u1 @ np.diag(s1**(0.5)) @ u1.T @ A2_white
A2_mixed_color = A2_mixed_color.reshape((channels,hw,hw))

diffA1 = np.mean((A1_mixed_color-A1)**2)
diffA2 = np.mean((A2_mixed_color-A2)**2)

# show originals
# and colored versions
G_A1 = show_gram(A1,'A1')
_ = show_gram(A1_white.reshape((channels,hw,hw)),'A1_white')
G_A1asA2 = show_gram(A1_mixed_color,'A1_mixed_color')

# show originals
# and colored versions
G_A1 = show_gram(A2,'A2')
_ = show_gram(A2_white.reshape((channels,hw,hw)),'A2_white')
G_A2asA1 = show_gram(A2_mixed_color,'A2_mixed_color')

print('A1 average content difference:', diffA1)
print('A2 average content difference:', diffA2)

print('A1 colored average Grammian difference:', np.mean((G_A2-G_A1asA2)**2))
print('A2 colored average Grammian difference:', np.mean((G_A1-G_A2asA1)**2))

## New implementation

Using the closed form solution of the transformation matrix is:

$$T = \Sigma_c^{-1/2}(\Sigma_c^{1/2}\Sigma_s\Sigma_c^{1/2})^{1/2}\Sigma_c^{-1/2}$$

Useful link: https://stats.stackexchange.com/questions/363425/raising-a-variance-covariance-matrix-to-a-negative-half-power

In [ ]:
def matrix_to_power(matrix, power):
    # Assuming real symmetric
    e_val, e_vec = np.linalg.eigh(matrix)
    lambda_ = np.diag(e_val**power)

    return e_vec@lambda_@e_vec.T

In [ ]:
def new_transform(content_img, style_img):
    sigma_c = np.cov(content_img)
    sigma_s = np.cov(style_img)

    simga_c_neg_half = matrix_to_power(sigma_c, -1/2)
    sigma_c_half = matrix_to_power(sigma_c, 1/2)

    inner = matrix_to_power(sigma_c_half@sigma_s@sigma_c_half, 1/2)

    transform_mat = simga_c_neg_half@inner@simga_c_neg_half
    return transform_mat@content_img

In [ ]:
A1.shape

In [ ]:
content = A1.reshape(channels, hw*hw)
style = A2.reshape(channels, hw*hw)

In [ ]:
content.shape


In [ ]:
colored = new_transform(content, style)
np.count_nonzero(np.isnan(colored))

In [ ]:
colored.shape

In [ ]:
_ = show_gram(colored.reshape((channels,hw,hw)),'A1 with A2 style')

In [ ]:
A2